In [1]:
import numpy as np
import math

# ---------------- 1. 路径模拟及敏感性信息 ----------------

def simulate_paths_with_derivs(s0, r, sigma, T, M, N, seed=42):
    """
    模拟几何布朗运动路径，并同时计算每个时间步 S_k 对 S0 与 sigma 的偏导。

    S_{k+1} = S_k * exp((r - 0.5*sigma^2)*dt + sigma*sqrt(dt)*Z_k)

    返回:
      S:         (M+1, N) 每条路径的价格
      dS_dS0:    (M+1, N) 每条路径中 S_k 对 S0 的偏导
      dS_dsigma: (M+1, N) 每条路径中 S_k 对 sigma 的偏导
    """
    np.random.seed(seed)
    dt = T / M

    S = np.zeros((M+1, N))
    dS_dS0 = np.zeros((M+1, N))
    dS_dsigma = np.zeros((M+1, N))

    S[0, :] = s0
    dS_dS0[0, :] = 1.0
    dS_dsigma[0, :] = 0.0

    for k in range(M):
        Z = np.random.standard_normal(N)
        drift = (r - 0.5 * sigma**2) * dt
        diff = sigma * math.sqrt(dt) * Z
        R = np.exp(drift + diff)

        S[k+1, :] = S[k, :] * R
        dS_dS0[k+1, :] = dS_dS0[k, :] * R

        # 对 sigma 求导： dR/dsigma = R * ( -sigma*dt + sqrt(dt)*Z )
        dR_dsigma = R * (-sigma*dt + math.sqrt(dt)*Z)
        dS_dsigma[k+1, :] = dS_dsigma[k, :] * R + S[k, :] * dR_dsigma

    return S, dS_dS0, dS_dsigma

# ---------------- 2. 对数桥存活概率及其偏导 ----------------

def barrier_prob_log(S_i, S_j, B, dt, sigma):
    """
    计算区间 [t_i, t_i+dt] 内不碰障碍 B 的概率及其对 S_i、S_j 与 sigma 的偏导：
      p = 1 - exp(-alpha), 其中 alpha = 2*(ln S_i - ln B)*(ln S_j - ln B) / (sigma^2 * dt)

    若 S_i 或 S_j 小于等于 B，则直接认为 p=0，导数=0。
    """
    if S_i <= B or S_j <= B:
        return 0.0, 0.0, 0.0, 0.0

    lnSi = np.log(S_i)
    lnSj = np.log(S_j)
    lnB  = np.log(B)

    alpha = 2.0 * (lnSi - lnB) * (lnSj - lnB) / (sigma**2 * dt)
    q = math.exp(-alpha)
    p = 1.0 - q

    # 对 S_i 的偏导： dp/dS_i = - d(exp(-alpha))/dS_i
    dq_dSi = -q * (2.0 * (lnSj - lnB) / (sigma**2 * dt * S_i))
    dp_dSi = -dq_dSi

    # 对 S_j 的偏导
    dq_dSj = -q * (2.0 * (lnSi - lnB) / (sigma**2 * dt * S_j))
    dp_dSj = -dq_dSj

    # 对 sigma 的偏导
    dalpha_dsigma = -4.0 * (lnSi - lnB) * (lnSj - lnB) / (sigma**3 * dt)
    dq_dsigma = -q * dalpha_dsigma
    dp_dsigma = -dq_dsigma

    return p, dp_dSi, dp_dSj, dp_dsigma

# ---------------- 3. 利用 Pathwise 方法计算价格、Delta 与 Vega ----------------

def barrier_call_pathwise(S, dS_dS0, dS_dsigma, K, B, r, T, sigma):
    """
    对每条路径计算下敲出看涨期权的贴现 payoff：
      payoff = e^{-rT} * (S_M - K)^+ * ∏_{i=0}^{M-1} p_i,
    并利用乘积法则对 payoff 关于 S0 与 sigma 求导，得到 pathwise 估计的 Delta 与 Vega。

    返回:
      price: 期权价格估计
      delta: Delta 估计 (对 S0 求导)
      vega:  Vega 估计 (对 sigma 求导)
    """
    M, N = S.shape[0]-1, S.shape[1]
    dt = T / M

    payoffs = np.zeros(N)
    dPdS0_arr = np.zeros(N)
    dPdSig_arr = np.zeros(N)
    discount = math.exp(-r*T)

    for j in range(N):
        pis = []       # 存储每步的 p_i
        dpis_dS0 = []  # 存储每步 p_i 对 S0 的链式偏导
        dpis_dSig = [] # 存储每步 p_i 对 sigma 的链式偏导

        survival_prod = 1.0
        # 对每个区间求 p_i 及其对敏感性的贡献
        for i in range(M):
            Si = S[i, j]
            Sip1 = S[i+1, j]
            p_i, dpdSi, dpdSip1, dpdSig = barrier_prob_log(Si, Sip1, B, dt, sigma)
            survival_prod *= p_i

            # 利用链式法则： dp_i/dS0 = (dp_i/dS_i)* (dS_i/dS0) + (dp_i/dS_{i+1})*(dS_{i+1}/dS0)
            dS_i_dS0   = dS_dS0[i, j]
            dS_ip1_dS0 = dS_dS0[i+1, j]
            dp_i_dS0   = dpdSi * dS_i_dS0 + dpdSip1 * dS_ip1_dS0

            # 对 sigma： dp_i/dsigma = (dp_i/dS_i)* (dS_i/dsigma) + (dp_i/dS_{i+1})*(dS_{i+1}/dsigma) + dp_i/dsigma
            dS_i_dSig   = dS_dsigma[i, j]
            dS_ip1_dSig = dS_dsigma[i+1, j]
            dp_i_dSigma = dpdSi * dS_i_dSig + dpdSip1 * dS_ip1_dSig + dpdSig

            pis.append(p_i)
            dpis_dS0.append(dp_i_dS0)
            dpis_dSig.append(dp_i_dSigma)

            # 若某步已敲出，则后续乘积为0，直接退出循环
            if survival_prod == 0.0:
                break

        SM = S[M, j]
        payoff_j = 0.0
        if SM > K:
            payoff_j = discount * (SM - K) * survival_prod
        payoffs[j] = payoff_j

        # 对路径 payoff 关于 S0 的导数：分两部分
        # (1) 终值部分：若 SM > K, 则 ∂(S_M-K)^+ /∂S0 = dS_dS0[M,j]
        # (2) 存活概率部分：利用对乘积求导展开，得到 Σ_i [ (dp_i/dS0) * ∏_{k≠i} p_k ]
        if payoff_j > 0.0:
            dPayoff_dS0 = discount * dS_dS0[M, j] * survival_prod
            SMminusK = SM - K
            prod_all = np.prod(pis) if len(pis) > 0 else 1.0
            partial_sum = 0.0
            for i in range(len(pis)):
                if pis[i] != 0.0:
                    partial_sum += dpis_dS0[i] * (prod_all / pis[i])
            dPayoff_dS0 += discount * SMminusK * partial_sum
            dPdS0_arr[j] = dPayoff_dS0
        else:
            dPdS0_arr[j] = 0.0

        # 同理，对 sigma 求导
        if payoff_j > 0.0:
            dPayoff_dSigma = discount * dS_dsigma[M, j] * survival_prod
            SMminusK = SM - K
            prod_all = np.prod(pis) if len(pis) > 0 else 1.0
            partial_sum = 0.0
            for i in range(len(pis)):
                if pis[i] != 0.0:
                    partial_sum += dpis_dSig[i] * (prod_all / pis[i])
            dPayoff_dSigma += discount * SMminusK * partial_sum
            dPdSig_arr[j] = dPayoff_dSigma
        else:
            dPdSig_arr[j] = 0.0

    price = np.mean(payoffs)
    delta = np.mean(dPdS0_arr)
    vega  = np.mean(dPdSig_arr)
    return price, delta, vega

# ---------------- 4. 主程序调用 ----------------

def main():
    # 参数设置
    s0    = 100.0
    K     = 110.0
    B     = 90.0
    r     = 0.05
    sigma = 0.5
    T     = 1.0
    M     = 500    # 时间步数（建议足够多以减小离散化误差）
    N     = 10000  # 模拟路径数

    # 模拟路径及敏感性信息
    S, dS_dS0, dS_dsigma = simulate_paths_with_derivs(s0, r, sigma, T, M, N)

    # 利用 pathwise 方法计算价格、Delta 与 Vega
    price, delta, vega = barrier_call_pathwise(S, dS_dS0, dS_dsigma, K, B, r, T, sigma)

    print("下敲出看涨期权 (Pathwise) 结果：")
    print("  Price :", price)
    print("  Delta :", delta)
    print("  Vega  :", vega)

if __name__ == "__main__":
    main()


下敲出看涨期权 (Pathwise) 结果：
  Price : 8.378607494557187
  Delta : 0.8421116840198639
  Vega  : 4.085153175874801
